# Creando DataFrames

In [22]:
from pyspark.sql import SparkSession

from pyspark.sql.types import (StructType, StructField,
                               IntegerType, StringType,
                               DoubleType, DateType)

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [23]:
# Crear RDD
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2))
rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]


## Crear Dataframe

In [24]:
df = rdd.toDF(['numero', 'cudrado'])

In [25]:
# esquema del dataframe
df.printSchema()

root
 |-- numero: long (nullable = true)
 |-- cudrado: long (nullable = true)



In [26]:
# visualizar los datos
df.show(10)

+------+-------+
|numero|cudrado|
+------+-------+
|     0|      0|
|     1|      1|
|     2|      4|
|     3|      9|
|     4|     16|
|     5|     25|
|     6|     36|
|     7|     49|
|     8|     64|
|     9|     81|
+------+-------+



### Crear un DataFrame a partir de un schema

In [27]:
# Crear un DataFrame a partir de un RDD con esquema

rdd1 = sc.parallelize(
    [(1, 'Jose', 35.5),
     (2, 'Teresa', 54.3),
     (3, 'Katia', 12.7)
     ])

Primera forma de crear un esquema

In [28]:
# Crear esquema
esquema1 = StructType(
    [
     StructField('id', IntegerType(), True),
     StructField('nombre', StringType(), True),
     StructField('saldo', DoubleType(), True)
    ]
)


In [29]:
df1 = spark.createDataFrame(rdd1, schema=esquema1)

df1.printSchema()

df1.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



Segunda forma

In [30]:
esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [31]:

df2 = spark.createDataFrame(rdd1, schema=esquema2)

df2.printSchema()

df2.show()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



### Crear un DataFrame a partir de un rango de números

In [32]:
spark.range(5).toDF('id').show()

spark.range(3, 15).toDF('id').show()

spark.range(0, 20, 2).toDF('id').show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

+---+
| id|
+---+
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+

+---+
| id|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 18|
+---+



## Crear DataFrames a partir de fuentes de datos en la práctica

In [33]:
# directorio donde estan los datos
path = 'files/data/'

### texto txt

In [34]:
# Crear un DataFrame mediante la lectura de un archivo de texto
df = spark.read.text(path+'dataTXT.txt')

df.show()


+--------------------+
|               value|
+--------------------+
|Estamos en el cur...|
|En este capítulo ...|
|En esta sección e...|
|y en este ejemplo...|
+--------------------+



In [35]:
# Mostrar los datos sin truncar la informacion
df.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



### CSV

In [36]:
# Crear un DataFrame mediante la lectura de un archivo csv
# los crea con nombres de columnas arbitrarios
df1 = spark.read.csv(path+'dataCSV.csv')

df1.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|        _c0|          _c1|                 _c2|                 _c3|        _c4|                 _c5|                 _c6|    _c7|   _c8|     _c9|         _c10|                _c11|             _c12|            _c13|                _c14|                _c15|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|vid

In [37]:
# Agregando los nombres de las columnas
df1 = spark.read.option('header', 'true').csv(path+'dataCSV.csv')

df1.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [38]:
# Leer un archivo de texto con un delimitador diferente

df2 = spark.read.option('header', 'true').option('delimiter', '|').csv(
    path+'dataTab.txt')

df2.show()


+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



### JSON

In [39]:
# Crear un DataFrame a partir de un json proporcionando un schema

json_schema = StructType(
    [
     StructField('color', StringType(), True),
     StructField('edad', IntegerType(), True),
     StructField('fecha', DateType(), True),
     StructField('pais', StringType(), True)
    ]
)

df4 = spark.read.schema(json_schema).json(path+'dataJSON.json')

df4.show()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|NULL|2021-02-21|  MX|
| azul|NULL|2021-06-10|  CA|
|verde|NULL|2020-06-02|  US|
+-----+----+----------+----+



In [40]:
# estructura del esquema
df4.printSchema()

root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



### parquet

In [41]:
# Crear un DataFrame a partir de un archivo parquet
df5 = spark.read.parquet(path+'dataPARQUET.parquet')

df5.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [42]:
# Otra alternativa para leer desde una fuente de datos parquet en este caso
df6 = spark.read.format('parquet').load(path+'dataPARQUET.parquet')

df6.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [21]:
sc.stop()